In [1]:
import tensorflow as tf
from keras import layers, models
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import keras.utils as image
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def create_augmented_images(input_directory, output_directory, augmentation_params, num_augmentations=5):
    
    # Create an ImageDataGenerator with the specified augmentations
    datagen = image.ImageDataGenerator(**augmentation_params)

    # Get a list of all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f))]

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Loop through each image file
    for _ in image_files:
        img_path = os.path.join(input_directory, _)

        # Load the image
        img = image.load_img(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Generate augmented images
        for i, _ in enumerate(datagen.flow(x, batch_size=1, save_to_dir=output_directory, save_prefix=f'aug_{i}', save_format='jpeg')):
            if i >= num_augmentations:
                break


In [3]:
# input_directory = "./Val/ad/"
# output_directory = "./Val/ad_aug/"
augmentation_params = {
    'rotation_range': 40,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True,
}

#create_augmented_images(input_directory, output_directory, augmentation_params)


In [4]:

# input_directory = "./Train/ad/"
# output_directory = "./Train/ad_aug/"
augmentation_params = {
    'rotation_range': 40,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True,
}

#create_augmented_images(input_directory, output_directory, augmentation_params)


In [5]:
# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./Train/Train_aug', 
                                                    target_size=(200, 200),
                                                    batch_size=32,
                                                    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('./Val/Val_aug',
                                                              target_size=(200, 200),
                                                              batch_size=32,
                                                              class_mode='binary')




Found 330 images belonging to 2 classes.
Found 330 images belonging to 2 classes.


In [6]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))


for layer in base_model.layers:
    layer.trainable = False


model_0 = models.Sequential([
    base_model,
    layers.Flatten(name='flatten'),
    layers.Dense(512, activation='relu', name='dense'),
    layers.Dense(1, activation='sigmoid', name='dense_1')
])

# Compile the model
model_0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model_0.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 512)               9437696   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 24,152,897
Trainable params: 9,438,209
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
# Train the model
history = model_0.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator)


Epoch 1/20


In [ ]:
model_0.save('model_0.h5')

In [ ]:
# Load the trained model
model_1 = load_model("model_0.h5")

In [ ]:
def classify_image(img_path):
    img = image.load_img(img_path, target_size=(200, 200))
    img_array = image.img_to_array(img)
    img_array = img_array.reshape((1, 200, 200, 3))
    img_array /= 255.0


    prediction = model_1.predict(img_array)


    confidence_score = prediction[0][0]


    score = int(round(confidence_score))


    classification = "Ad creatives" if  score == 0 else "Non-Ad creatives"

    return score, classification



In [ ]:
img_path = "./henz_ketchup.png"
label, classification = classify_image(img_path)

print(label)
print(classification)


1/1 [==============================] - 0s 90ms/step
0
Ad creatives
